In [1]:
! pip list

Package           Version
----------------- -----------
asttokens         2.4.1
colorama          0.4.6
comm              0.2.2
debugpy           1.8.1
decorator         5.1.1
exceptiongroup    1.2.0
executing         2.0.1
ipykernel         6.29.4
ipython           8.23.0
jedi              0.19.1
jupyter_client    8.6.1
jupyter_core      5.7.2
matplotlib-inline 0.1.6
nest-asyncio      1.6.0
numpy             1.26.4
packaging         24.0
pandas            2.2.2
parso             0.8.4
pip               24.0
platformdirs      4.2.0
prompt-toolkit    3.0.43
psutil            5.9.8
pure-eval         0.2.2
Pygments          2.17.2
python-dateutil   2.9.0.post0
pytz              2024.1
pywin32           306
pyzmq             25.1.2
setuptools        65.5.0
six               1.16.0
stack-data        0.6.3
tornado           6.4
traitlets         5.14.2
typing_extensions 4.11.0
tzdata            2024.1
wcwidth           0.2.13


In [2]:
! pip install pandas

In [3]:
import pandas as pd
import os

Go back one level on the working directory to have to full folder

In [4]:
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

c:\Users\USUARIO\Desktop\GITHUB Projects\video_games_sales\notebooks
c:\Users\USUARIO\Desktop\GITHUB Projects\video_games_sales


# Reading CSV file as pandas dataframe

In [5]:
data_root="./data"
raw_data= pd.read_csv(data_root+"/raw/vgsales_.csv",engine="python")

In [6]:
raw_data.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,NaN,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,NaN,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


# Data Quality

## Checking Dtypes

Checking data types, User_Score is a numeric value but pandas is reading it as object, mainly to a value 'tbd' which is giving a bad read this data type is going to be changed

In [7]:
raw_data.dtypes

Name                object
Platform            object
Year_of_Release    float64
Genre               object
Publisher           object
NA_Sales           float64
EU_Sales           float64
JP_Sales           float64
Other_Sales        float64
Global_Sales       float64
Critic_Score       float64
Critic_Count       float64
User_Score          object
User_Count         float64
Developer           object
Rating              object
dtype: object

In [8]:
# using coerce to transform string values into nan
raw_data["User_Score"]=pd.to_numeric(raw_data["User_Score"], errors='coerce')

In [9]:
raw_data["User_Score"].unique()

array([8. , nan, 8.3, 8.5, 6.6, 8.4, 8.6, 7.7, 6.3, 7.4, 8.2, 9. , 7.9,
       8.1, 8.7, 7.1, 3.4, 5.3, 4.8, 3.2, 8.9, 6.4, 7.8, 7.5, 2.6, 7.2,
       9.2, 7. , 7.3, 4.3, 7.6, 5.7, 5. , 9.1, 6.5, 8.8, 6.9, 9.4, 6.8,
       6.1, 6.7, 5.4, 4. , 4.9, 4.5, 9.3, 6.2, 4.2, 6. , 3.7, 4.1, 5.8,
       5.6, 5.5, 4.4, 4.6, 5.9, 3.9, 3.1, 2.9, 5.2, 3.3, 4.7, 5.1, 3.5,
       2.5, 1.9, 3. , 2.7, 2.2, 2. , 9.5, 2.1, 3.6, 2.8, 1.8, 3.8, 0. ,
       1.6, 9.6, 2.4, 1.7, 1.1, 0.3, 1.5, 0.7, 1.2, 2.3, 0.5, 1.3, 0.2,
       0.6, 1.4, 0.9, 1. , 9.7])

## Ratings must be standardize given rating has changes since 1980 to the present

K-A should be change to E; EC should be changed to E

In [10]:
raw_data["Rating"].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

In [11]:
raw_data["Rating"]=raw_data["Rating"].map({
    'E':'E', 'nan':'nan', 'M':'M', 'T':'T', 'E10+':'E10+', 'K-A':'E', 'AO':'AO', 'EC':'E', 'RP':'RP'
})

In [12]:
raw_data["Rating"].unique()

array(['E', nan, 'M', 'T', 'E10+', 'AO', 'RP'], dtype=object)

## Checking null/nan values
After checking for nan/null values we have ahigh amount of missing values given the data shape (16719), specially on the critics and user count, this is mainly given to the year of release of most the games in the data base, some coming from the 80s were a centralized based of scores was not in use.

In [13]:
raw_data.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales            871
EU_Sales           1199
JP_Sales              0
Other_Sales        1378
Global_Sales        991
Critic_Score       8582
Critic_Count       8582
User_Score         9129
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

## Describe of the numeric variables

There are variables in zero and are congruent, given some games never were sold on the specific region, or never were grade by any critics

In [14]:
raw_data.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count
count,16450.000000,15848.000000,15520.000000,16719.000000,15341.000000,15728.000000,8137.000000,8137.000000,7590.000000,7590.000000
mean,2006.487356,0.263579,0.146210,0.077602,0.046820,0.531498,68.967679,26.360821,7.125046,162.229908
std,5.878995,0.822121,0.513323,0.308818,0.175121,1.539597,13.938165,18.980495,1.500006,561.282326
min,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000,13.000000,3.000000,0.000000,4.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000,60.000000,12.000000,6.400000,10.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000,71.000000,21.000000,7.500000,24.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,0.470000,79.000000,36.000000,8.200000,81.000000
max,2020.000000,41.360000,28.960000,10.220000,10.570000,82.530000,98.000000,113.000000,9.700000,10665.000000


# Export Clean data as csv

In [15]:
raw_data.to_csv(data_root+"/preprocessed/vg_sales_clean.csv",index=False)